In [1]:
# -*- coding:utf:8 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import pytorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
# 输入dancing用一个维度为3的向量表示， 隐藏层的一个维度为3，仅有一层神经元
# 此时，神经层的详细结构还没确定，仅仅是说这个网络可以接受[seq_len, batch_size, 3]的数据输入
lstm = nn.LSTM(3,3)
# 构造一个由5个dancing组成的句子，构造出来的句子向量为[5,1,3]，也就是明确告诉网络：一条句子由5个单词组成
inputs = [torch.randn(1,3) for _ in range(5)]
print('Inputs:', inputs)
# 初始化隐藏状态
hidden = (torch.randn(1,1,3), torch.randn(1,1,3))
print('hidden:', hidden)

for i in inputs:
    # 将序列的元素逐个输入到LSTM，这里的view是把输入放到第三维，看起来有点怪
    # 这里是固定的格式，无论是什么形式的数据都必须放到这个维度
    out, hidden = lstm(i.view(1,1,-1), hidden)
print('out1:', out)
print('hidden2:', hidden)

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1,1,3), torch.randn(1,1,3))
out, hidden = lstm(inputs, hidden)
print('out2:', out)
print('hidden3:', hidden)

Inputs: [tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]
hidden: (tensor([[[ 0.5438, -0.4057,  1.1341]]]), tensor([[[-1.1115,  0.3501, -0.7703]]]))
out1: tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>)
hidden2: (tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward>))
out2: tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
hidden3: (tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ('The dog ate the apple'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ('Everybody read that book'.split(), ['NN', 'V', 'DET', 'NN'])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)
tag_to_ix = {
    'DET': 0,
    'NN': 1,
    'V': 2
}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [4]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # the LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # the linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
# train the model

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# see what the scores are before training
# note that element i,j of the output is the score for tag j for word i
# here we don't need to train, so the code is wrapped in torch.no_grad()

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

# 实际情况下不会训练300个周期
for epoch in range(300):
    for sentence, tags in training_data:
        # step 1. pytorch会累加梯度。每次训练前需要清空梯度值
        model.zero_grad()

        # step 2. 准备网络输入，将其变为词索引的Tensor类型数据
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # step 3. 前向传播
        tag_scores = model(sentence_in)

        # step 4. 计算损失和梯度值，通过调用optimizer.step()来更新梯度
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# see what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # the sentence is 'the dog ate the apple'. i,j corresponds to score for tag j 
    # for word i. the predicted tag is the maximum scoring tag.
    # here, wecan see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # which is DET NOUN VERB DET NOUN, the correct sequence
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])
